# Perform Branch and Cut Algorithm 
Use built in optimizaiton functions to use cuts prioritized by trained Neural UCB model

In [1]:
import pyscipopt as pyopt
import branch_cut
import model
import pulp
import os
class arguments:
    def __init__(self):
        self.model_path = 'shell.ai/torch_model'
        self.ucb_on = True
        self.ucb_val_on = True

args = arguments()

#### Load Trained Model

In [2]:
scoring = model.NeuralUCB('Neural_UCB', .1, .1, .1, args)

In [3]:
variables, instance = pulp.LpProblem.fromMPS('total_cost.mps')

#### Score separtors and pick best one
For each separator:
1. Load new optimization model
2. Add separator
3. Configure model with objective function, constraints and separator
4. Score Model with loaded separator

Choose model with best separator to run mip on at next step

In [4]:
best_separator_score = -1
model_path = 'model'

cuts = os.listdir(model_path)
for separator in cuts:
    final_model = pyopt.Model('Shell_ai')
    sepa = pyopt.scip.Sepa()
    final_model.includeSepa(sepa, model_path, separator)
    configured_sepa, configured_model = branch_cut.configure_model(final_model, instance, sepa, separator)
    score = scoring.getActionsScores(configured_model)
    print("separator type: {}, score: {}".format(separator, score))
    if score > best_separator_score:
        best_separator_score = score
        best_separator = separator

print("\nBest Beparator {}".format(best_separator))

separator type: clique, score: 0.789103837
separator type: zerohalf, score: 0.0013449886
separator type: gomory, score: 0.370551215092

Best Beparator clique


In [7]:
final_model = pyopt.Model('Shell_ai')
final_model.includeSepa(pyopt.scip.Sepa(), model_path, best_separator)
configured_sepa, configured_model = branch_cut.configure_model(final_model, instance, sepa, best_separator)

solution = branch_cut.solve(instance, configured_model, time_limit=30)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4dfcab548c364ccdb5edeaeb68ccbad2-pulp.mps -sec 30 -timeMode elapsed -branch -printingOptions all -solution /tmp/4dfcab548c364ccdb5edeaeb68ccbad2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7433 COLUMNS
At line 282698 RHS
At line 290127 BOUNDS
At line 335304 ENDATA
Problem MODEL has 7428 rows, 45176 columns and 139736 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 30
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.78355e+08 - 1.49 seconds
Cgl0003I 0 fixed, 7096 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 7428 rows, 45176 columns (45176 integer (0 of which binary)) and 139736 elements
Cbc0031I 256 added rows had average density of 170.65625
Cbc0013I At